In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dgl
from dgl.nn import ChebConv
import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np
import pandas as pd
import numpy.linalg as alg
import os
from tqdm import tqdm
from functools import reduce
import argparse


def node_connects_cluster(node):
    return set(map(lambda x: inverse_cluster_dict[x], list(g[node]))).union(set([inverse_cluster_dict[node]]))


def po_linear_model(graph, alpha=1, beta=1, sigma=0.1, gamma=2):    
    for i in graph.nodes:        
        graph.nodes[i]["y"] = alpha + beta * graph.nodes[i]["z"] + sigma * np.random.normal() +  gamma * sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]])/graph.degree[i]  

def po_multiplicative_model(graph, alpha=1, sigma=0.1, delta=1, gamma=2): 
    for i in graph.nodes:
        graph.nodes[i]["y"] = ( (alpha + sigma * np.random.normal()) * graph.degree[i]/avg_deg )  *  (1 + delta * graph.nodes[i]["z"] + gamma * sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]]) / len(graph[i]) )




/root/miniconda3/envs/myconda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

path = 'Dataset/socfb-Stanford3.mtx'

df = pd.read_table(path, skiprows=1, names = ["source", "target"], sep=" ")
g = nx.from_pandas_edgelist(df)

# calculate basic elements
num_nodes = g.number_of_nodes()
num_edges = g.number_of_edges()
degs = [g.degree[i] for i in g.nodes]
avg_deg = sum(degs)/len(degs)

res = 10


# clustering
# generally, we fix the outcome of clustering
clusters = nx_comm.louvain_communities(g, seed = 10, resolution=res)
clusters = sorted(clusters, key = len, reverse=True)
cluster_sizes = list(map(len, clusters))
num_cluster = len(clusters)

# dict: from node to its cluster
inverse_cluster_dict = {
    node: cl for cl in range(num_cluster) for node in clusters[cl]
}

# dict: from node to its connected cluster
node_to_connected_clusters = {
    node: node_connects_cluster(node) for node in range(1, num_nodes + 1)
}


for i in g.nodes:
    g.nodes[i]["n_cl"] = len(node_to_connected_clusters[i])
    g.nodes[i]["deg"] = g.degree[i]

rev_cluster_sizes = cluster_sizes[::-1]
rev_clusters = clusters[::-1]
rev_cluster_sizes = np.array(rev_cluster_sizes)


In [3]:
dgl_G = dgl.from_networkx(g)

In [4]:
ramps = [0.02, 0.05, 0.1, 0.25, 0.5]
repeat_num = 1000

In [5]:

class GCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = dgl.nn.ChebConv(2, 16, 2)
        self.conv2 = dgl.nn.ChebConv(16, 16, 1)
        self.conv3 = dgl.nn.ChebConv(16, 1, 1)
        
    def forward(self, g, features):
        x = self.conv1(g, features)
        x = self.conv2(g, x)
        x = self.conv3(g, x)
        return x
        

In [ ]:
# unit-level cr

GATE_hat_list = np.zeros((repeat_num, len(ramps)))

     
for seed in tqdm(range(repeat_num)):
    
    np.random.seed(seed)     
    rollout_index = np.random.uniform(0, 1, size=(num_nodes,))   
    
    for num_step in range(len(ramps)): 
        print(f"Seed: {seed}")
        torch.manual_seed(2)
        p_list = ramps[num_step:]
        g_feat_list = []
        y_list = []
        
        for p in p_list:
            z = (rollout_index < p)            
            nx.set_node_attributes(g, 0, "z")
            nx.set_node_attributes(g, {unit:1 for unit in range(num_nodes) if z[unit]}, "z")
            po_linear_model(g, gamma = 1)
            
            g_feat_list.append(torch.tensor([[g.nodes[n]['z'], g.nodes[n]['deg']] for n in g.nodes], dtype=torch.float))
            y_list.append(torch.tensor([[g.nodes[n]['y']] for n in g.nodes], dtype=torch.float).reshape(-1))
    
        # Instantiate the model and optimizer
        model = GCN()    
        optimizer = torch.optim.Adam(model.parameters(), lr=0.004)
            
        # Train the model
        for epoch in range(400):
            for i in range(len(p_list)):
                model.train()
                optimizer.zero_grad()
                out = model(dgl_G, g_feat_list[i])
                loss = F.mse_loss(out.squeeze(), y_list[i])  
                loss.backward()
                optimizer.step()
            
                if epoch % 50 == 0:
                    print(f'Epoch {epoch}, Treat Prop: {p_list[i]:.2f}, Loss: {loss:.4f}')
    
        print(loss.item())
        g_feat = g_feat_list[0]
    
        g_feat[:,0] = 0
        global_control_pred = model(dgl_G, g_feat).detach().numpy()
    
        g_feat[:,0] = 1
        global_treat_pred = model(dgl_G, g_feat).detach().numpy()
    
        GATE_hat = global_treat_pred.mean() - global_control_pred.mean()
    
        GATE_hat_list[seed, num_step] = GATE_hat
        
    torch.save(GATE_hat_list, "Result/Rollout_Ber_unit_res10_final.pkl")        


  0%|          | 0/1000 [00:00<?, ?it/s]

Seed: 0


/root/miniconda3/envs/myconda/lib/python3.10/site-packages/dgl/nn/pytorch/conv/chebconv.py:108: DGLWarning: lambda_max is not provided, using default value of 2.  Please use dgl.laplacian_lambda_max to compute the eigenvalues.
  dgl_warning(


Epoch 0, Treat Prop: 0.02, Loss: 8.0117
Epoch 0, Treat Prop: 0.05, Loss: 2.3802
Epoch 0, Treat Prop: 0.10, Loss: 1.2685
Epoch 0, Treat Prop: 0.25, Loss: 1.9776
Epoch 0, Treat Prop: 0.50, Loss: 3.4543
Epoch 50, Treat Prop: 0.02, Loss: 0.1053
Epoch 50, Treat Prop: 0.05, Loss: 0.1390
Epoch 50, Treat Prop: 0.10, Loss: 0.0924
Epoch 50, Treat Prop: 0.25, Loss: 0.1570
Epoch 50, Treat Prop: 0.50, Loss: 0.4368
Epoch 100, Treat Prop: 0.02, Loss: 0.0218
Epoch 100, Treat Prop: 0.05, Loss: 0.0305
Epoch 100, Treat Prop: 0.10, Loss: 0.0249
Epoch 100, Treat Prop: 0.25, Loss: 0.0444
Epoch 100, Treat Prop: 0.50, Loss: 0.0660
Epoch 150, Treat Prop: 0.02, Loss: 0.0269
Epoch 150, Treat Prop: 0.05, Loss: 0.0246
Epoch 150, Treat Prop: 0.10, Loss: 0.0252
Epoch 150, Treat Prop: 0.25, Loss: 0.0485
Epoch 150, Treat Prop: 0.50, Loss: 0.0611
Epoch 200, Treat Prop: 0.02, Loss: 0.0174
Epoch 200, Treat Prop: 0.05, Loss: 0.0214
Epoch 200, Treat Prop: 0.10, Loss: 0.0279
Epoch 200, Treat Prop: 0.25, Loss: 0.0379
Epoch 2

  0%|          | 1/1000 [00:34<9:35:56, 34.59s/it]

...

Seed: 243
Epoch 0, Treat Prop: 0.02, Loss: 8.0369
Epoch 0, Treat Prop: 0.05, Loss: 2.3718
Epoch 0, Treat Prop: 0.10, Loss: 1.2588
Epoch 0, Treat Prop: 0.25, Loss: 1.9418
Epoch 0, Treat Prop: 0.50, Loss: 3.4123
Epoch 50, Treat Prop: 0.02, Loss: 0.0958
Epoch 50, Treat Prop: 0.05, Loss: 0.1319
Epoch 50, Treat Prop: 0.10, Loss: 0.0822
Epoch 50, Treat Prop: 0.25, Loss: 0.1431
Epoch 50, Treat Prop: 0.50, Loss: 0.3806
Epoch 100, Treat Prop: 0.02, Loss: 0.0241
Epoch 100, Treat Prop: 0.05, Loss: 0.0269
Epoch 100, Treat Prop: 0.10, Loss: 0.0242
Epoch 100, Treat Prop: 0.25, Loss: 0.0439
Epoch 100, Treat Prop: 0.50, Loss: 0.0631
Epoch 150, Treat Prop: 0.02, Loss: 0.0332
Epoch 150, Treat Prop: 0.05, Loss: 0.0239
Epoch 150, Treat Prop: 0.10, Loss: 0.0271
Epoch 150, Treat Prop: 0.25, Loss: 0.0519
Epoch 150, Treat Prop: 0.50, Loss: 0.0607
Epoch 200, Treat Prop: 0.02, Loss: 0.0326
Epoch 200, Treat Prop: 0.05, Loss: 0.0238
Epoch 200, Treat Prop: 0.10, Loss: 0.0268
Epoch 200, Treat Prop: 0.25, Loss: 0.05

 24%|██▍       | 244/1000 [2:13:30<6:38:32, 31.63s/it]

0.03593714162707329
Seed: 244
Epoch 0, Treat Prop: 0.02, Loss: 8.0158
Epoch 0, Treat Prop: 0.05, Loss: 2.3846
Epoch 0, Treat Prop: 0.10, Loss: 1.2696
Epoch 0, Treat Prop: 0.25, Loss: 1.9747
Epoch 0, Treat Prop: 0.50, Loss: 3.4353
Epoch 50, Treat Prop: 0.02, Loss: 0.1032
Epoch 50, Treat Prop: 0.05, Loss: 0.1398
Epoch 50, Treat Prop: 0.10, Loss: 0.0999
Epoch 50, Treat Prop: 0.25, Loss: 0.1813
Epoch 50, Treat Prop: 0.50, Loss: 0.4921
Epoch 100, Treat Prop: 0.02, Loss: 0.0236
Epoch 100, Treat Prop: 0.05, Loss: 0.0335
Epoch 100, Treat Prop: 0.10, Loss: 0.0244
Epoch 100, Treat Prop: 0.25, Loss: 0.0446
Epoch 100, Treat Prop: 0.50, Loss: 0.0670
Epoch 150, Treat Prop: 0.02, Loss: 0.0271
Epoch 150, Treat Prop: 0.05, Loss: 0.0242
Epoch 150, Treat Prop: 0.10, Loss: 0.0256
Epoch 150, Treat Prop: 0.25, Loss: 0.0479
Epoch 150, Treat Prop: 0.50, Loss: 0.0581
Epoch 200, Treat Prop: 0.02, Loss: 0.0324
Epoch 200, Treat Prop: 0.05, Loss: 0.0254
Epoch 200, Treat Prop: 0.10, Loss: 0.0275
Epoch 200, Treat Pr

 24%|██▍       | 245/1000 [2:14:15<7:27:34, 35.57s/it]

Seed: 245
Epoch 0, Treat Prop: 0.02, Loss: 8.0440
Epoch 0, Treat Prop: 0.05, Loss: 2.3878
Epoch 0, Treat Prop: 0.10, Loss: 1.2548
Epoch 0, Treat Prop: 0.25, Loss: 1.9215
Epoch 0, Treat Prop: 0.50, Loss: 3.3360
Epoch 50, Treat Prop: 0.02, Loss: 0.1305
Epoch 50, Treat Prop: 0.05, Loss: 0.1699
Epoch 50, Treat Prop: 0.10, Loss: 0.1001
Epoch 50, Treat Prop: 0.25, Loss: 0.1882
Epoch 50, Treat Prop: 0.50, Loss: 0.5423
Epoch 100, Treat Prop: 0.02, Loss: 0.0242
Epoch 100, Treat Prop: 0.05, Loss: 0.0291
Epoch 100, Treat Prop: 0.10, Loss: 0.0252
Epoch 100, Treat Prop: 0.25, Loss: 0.0437
Epoch 100, Treat Prop: 0.50, Loss: 0.0667
Epoch 150, Treat Prop: 0.02, Loss: 0.0289
Epoch 150, Treat Prop: 0.05, Loss: 0.0247
Epoch 150, Treat Prop: 0.10, Loss: 0.0248
Epoch 150, Treat Prop: 0.25, Loss: 0.0481
Epoch 150, Treat Prop: 0.50, Loss: 0.0597
Epoch 200, Treat Prop: 0.02, Loss: 0.0319
Epoch 200, Treat Prop: 0.05, Loss: 0.0239
Epoch 200, Treat Prop: 0.10, Loss: 0.0261
Epoch 200, Treat Prop: 0.25, Loss: 0.05